# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397807683005                   -0.90    5.2   62.4ms
  2   -8.400237723330       -2.61       -1.74    1.0   19.3ms
  3   -8.400404604187       -3.78       -2.97    1.5   20.1ms
  4   -8.400427862976       -4.63       -2.98    3.2   24.4ms
  5   -8.400427927672       -7.19       -3.02    1.0   19.3ms
  6   -8.400428149309       -6.65       -4.67    1.0   19.0ms
  7   -8.400428155819       -8.19       -4.42    2.8   45.7ms
  8   -8.400428156247       -9.37       -5.10    1.0   19.7ms
  9   -8.400428156275      -10.56       -6.60    1.0   19.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397806444056                   -0.90           5.2    604ms
  2   -8.400382161421       -2.59       -1.79   0.80    2.0    547ms
  3   -8.400422238155       -4.40       -3.02   0.80    1.0    202ms
  4   -8.400428116205       -5.23       -3.41   0.80    2.5   21.0ms
  5   -8.400428153460       -7.43       -4.55   0.80    1.0   17.0ms
  6   -8.400428156234       -8.56       -5.68   0.80    2.2   20.1ms
  7   -8.400428156277      -10.37       -6.31   0.80    3.0   59.4ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.030984982622                   -1.07    2.52s
  2   -1.342092463780        0.38       -0.69    125ms
  3   -4.333030614006        0.48       -0.41   87.5ms
  4   -5.886637770556        0.19       -0.47   44.6ms
  5   -7.477222243094        0.20       -0.75   44.6ms
  6   -7.962479251928       -0.31       -1.19   33.1ms
  7   -8.188689477827       -0.65       -1.44   58.5ms
  8   -8.261864208048       -1.14       -1.84   33.0ms
  9   -8.309430961835       -1.32       -1.99   32.9ms
 10   -8.340665172619       -1.51       -1.98   33

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397799642488                   -0.90    5.5   27.0ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427978011                   -1.78    25.3s
  2   -8.400428156277       -6.75       -4.02    2.64s
  3   -8.400428156277      -14.45       -7.81    118ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.54710688039219e-7
|ρ_newton - ρ_scfv| = 1.6032366028009072e-7
|ρ_newton - ρ_dm|   = 1.0346387872161044e-6
